In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
from fastai.dataloader import *
from fastai.dataset import *
from fastai.transforms import *
from fastai.models import *
from fastai.conv_learner import *

In [3]:
PATH = Path('../../ILSVRC/Data/CLS-LOC/')

In [4]:
from pathlib import Path

### Training model

In [5]:
arch = resnet152
tfms = tfms_from_model(arch, 256, aug_tfms=transforms_side_on)
bs = 128

In [6]:
# data = ImageClassifierData.from_csv(DIR, 'train1', DIR/TRAIN_CSV, tfms=tfms, bs=bs)
def get_data(sz,bs):
#     tfms = tfms_from_stats(stats, sz, aug_tfms=[RandomFlip()], pad=sz//8)
    return ImageClassifierData.from_paths(PATH, val_name='val', tfms=tfms, bs=bs)

In [7]:
data = get_data(256,bs)

In [8]:
m = ConvnetBuilder(resnet34, data.c, data.is_multi, data.is_reg)
# models = ConvnetBuilder(resnet34, data.c, data.is_multi, data.is_reg)
# m.model = network_to_half(m.model)
learner = ConvLearner(data, m)

In [16]:
# learner.float()

In [9]:
# float
%time learner.fit(0.5,1,cycle_len=.01)

101it [00:54,  1.85it/s, loss=3.89]                         

KeyboardInterrupt: 

In [23]:
learner.half()

In [24]:
%time learner.fit(0.5,1,cycle_len=.1)

  0%|          | 0/1001 [00:00<?, ?it/s]                       


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_monitor.py", line 62, in run
    for instance in self.tqdm_cls._instances:
  File "/home/paperspace/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



 25%|██▍       | 248/1001 [02:04<06:17,  1.99it/s, loss=2.65]


KeyboardInterrupt: 

### Training model 16

In [ ]:
from fp16utils import *

In [ ]:
arch = resnet34
tfms = tfms_from_model(arch, 256, aug_tfms=transforms_side_on)
bs = 128

In [ ]:
data = ImageClassifierData.from_csv(DIR, 'train1', DIR/TRAIN_CSV, tfms=tfms, bs=bs)

In [ ]:
m16 = ConvnetBuilder(resnet34, data.c, data.is_multi, data.is_reg)
m16.model = network_to_half(m16.model)
learner16 = ConvLearner(data, m16)

In [ ]:
learner16.model

In [ ]:
%time learner16.fit(0.5,1,cycle_len=1)

## Results

No speedup here.  
You can confirm that we are using v100 tensor cores by running this:  
`/usr/local/cuda/bin/nvprof --log-file nvprof_output.txt python profile_fp16_imagenet.py`